In [3]:
# ***Be sure to source setup.bash before launching jupyter server***
%matplotlib inline

import rospy
import rosbag
import numpy as np
import json
import os
import glob
import matplotlib.pyplot as plt
import tf_conversions
import scipy.stats as st

from matplotlib.patches import Ellipse

from custom_messages.msg import SimulationDataMsg

import sys
sys.path.append("/home/joshua/Documents/Uni/Year4/dissertation/catkin_ws/src/linefollow_gazebo/scripts/")
import CameraNetwork
import Path 
import RoadModel
import Tracer


In [4]:
def point_to_numpy(point):
    return np.array([point.x, point.y, point.z])
def vec_to_numpy(vec):
    return point_to_numpy(vec)
def quat_to_numpy(quat):
    return np.array([quat.x, quat.y, quat.z, quat.w])
def quat_to_rpy(quat):
    # note: need numpy quaternion
    return tf_conversions.transformations.euler_from_quaternion(quat)

In [5]:
experiment_defs_root = "defs"
experiment_gen_root = "gen"
specific_experiment_paths = [['calibration_circuit.json'], ['calibration_circuit_mirror.json']] # subbpath to only process 1 experiment dir or .json, not all at once
# specific_experiment_paths = [['straight_road_80m_centercam_pos3.json']]
if len(specific_experiment_paths) > 0:
    specific_experiments = [os.path.join(*p) for p in  specific_experiment_paths]


In [14]:
def get_bagfiles_for(experiment_path):
    if not experiment_path[-1].endswith('.json'):
        print("must provide .json file")
    gen_path = 'gen'
    if len(experiment_path) > 0:
        if len(experiment_path) > 1:
            gen_path = os.path.join(gen_path, os.path.join(*experiment_path[:-1]))
        exp_name = experiment_path[-1].split('.')[-2]
        gen_path = os.path.join(gen_path, exp_name)
    if os.path.isdir(gen_path):
        # collect all .bag files!
        bag_files = []
        for path, dirs, files in os.walk(gen_path):
            for f in files:
                if f.endswith('.bag'):
                    bag_file_path = os.path.join(path, f)
                    bag_files.append(bag_file_path)
        return bag_files
    else:
        print("{0} is not generated!".format(gen_path))


In [19]:
def get_bagfiles_for_set(experiment_path):
    # assume pass in a directory containing many .json files
    files = os.listdir(os.path.join(*experiment_path))
    
    bagfiles = []
    def_files = []
    for def_file in files:
        if not def_file.endswith('.json'):
            print("skipping: {0}".format(def_file))
            continue
        path = experiment_path + [def_file]
        def_files.append(path)
        bagfiles.append(get_bagfiles_for(path))
    
    
    return def_files, bagfiles
    

['gen/calibration_circuit/runner_0/run_0/sim_data.bag',
 'gen/calibration_circuit/runner_0/run_1/sim_data.bag',
 'gen/calibration_circuit/runner_1/run_2/sim_data.bag',
 'gen/calibration_circuit/runner_1/run_0/sim_data.bag',
 'gen/calibration_circuit/runner_1/run_1/sim_data.bag']

In [33]:
def sum_step_entropies(bag, max_steps):
    total_entropy = 0
    for i, msg in enumerate(bag):
        if i >= max_steps:
            break
        msg = msg.message
        ekf_cov = np.array(msg.ekf_odom.pose.covariance).reshape(6,6)[:3, :3]
        # have covariance truncated to x, y, theta only
        
        # diff. entropy = H(xt | z1:t, u1:t)
        differential_entropy = np.log(np.sqrt((2*np.pi*np.e)**(3) * np.linalg.det(ekf_cov)))
        
        total_entropy += differential_entropy
    
    return total_entropy
    

In [34]:
def calculate_mean_summed_entropy(bagfiles, max_steps):
    total, n = 0.0, 0
    for f in bagfiles:
        bag = rosbag.Bag(f)
        total += sum_step_entropies(bag, max_steps)
        n += 1
    
    return total/n

In [35]:
def get_min_num_in_files(bagfiles):
    min_n = 9999999
    for f in bagfiles:
        bag = rosbag.Bag(f)
        num_msgs = bag.get_message_count()
        if num_msgs < min_n:
            min_n = num_msgs
    return min_n

def get_min_number_of_steps(set_of_bagfiles):
    """ Takes list of lists of bagfiles, finds the minimum steps taken"""
    min_steps = 99999999
    for files in set_of_bagfiles:
        m = get_min_num_in_files(files)
        if m < min_steps:
            min_steps = m
    return min_steps

In [46]:
# for one example for now, wrap in function later
files = get_bagfiles_for(['straight_road_80m_centercam_pos3.json'])
# max_steps = get_min_number_of_steps([files])
max_steps = 141
print("Mean summed entropy from covariance matrices: {0}".format(calculate_mean_summed_entropy(files, max_steps)))
print(max_steps)

Mean summed entropy from covariance matrices: -1029.5184933
141


In [47]:
# for one example for now, wrap in function later
files = get_bagfiles_for(['straight_road_80m_centercam_pos2.json'])
max_steps = get_min_number_of_steps([files])
print("Mean summed entropy from covariance matrices: {0}".format(calculate_mean_summed_entropy(files, max_steps)))
print(max_steps)

Mean summed entropy from covariance matrices: -1010.33328212
141


In [48]:
# for one example for now, wrap in function later
files = get_bagfiles_for(['straight_road_80m_centercam_pos1.json'])
max_steps = get_min_number_of_steps([files])
max_steps=141
print("Mean summed entropy from covariance matrices: {0}".format(calculate_mean_summed_entropy(files, max_steps)))
print(max_steps)

Mean summed entropy from covariance matrices: -953.132931825
141
